In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from aymurai.utils.display import DocRender
from aymurai.datasets.ar_juz_pcyf_10.annotations import (
    ArgentinaJuzgadoPCyF10LabelStudioAnnotations,
)
from aymurai.utils.misc import get_element

render = DocRender()
docs = ArgentinaJuzgadoPCyF10LabelStudioAnnotations(
    "/resources/data/restricted/annotations/20221130-bis/"
).data

sample = docs[:10]

print(len(docs))


# test pipeline

In [ ]:
from aymurai.pipeline import AymurAIPipeline
from aymurai.models.flair.core import FlairModel
from aymurai.models.flair.utils import FlairTextNormalize
from aymurai.transforms.entity_subcategories.regex import RegexSubcategorizer
from aymurai.transforms.datetime_formatter.core import DatetimeFormatter


config = {
    "preprocess": [
        (FlairTextNormalize, {}),
    ],
    "models": [
        (
            FlairModel,
            {
                "basepath": "/resources/pipelines/examples/flair-simple/FlairModel",
                "split_doc": False,
                "device": "cpu",
            },
        )
    ],
    "postprocess": [
        (RegexSubcategorizer, {}),
        (DatetimeFormatter, {}),
    ],
    # "multiprocessing": {},
    "use_cache": False,
    # 'log_level': 'debug'
}

pipeline = AymurAIPipeline(config)

In [ ]:
doc = sample[1:2]
pars = [{'path': 'empty', 'data': {'doc.text': par}} for par in doc[0]['data']['doc.text'].splitlines()]
# pars

In [ ]:
for par in pars:
    # if not '10:00' in par['data']['doc.text']:
        # continue
    preprocessed = pipeline.preprocess([par])
    # predicted = pipeline.predict(preprocessed)
    predicted = pipeline.predict_single(preprocessed[0])
    postprocessed = pipeline.postprocess([predicted])
    item = postprocessed[0]

    ents = get_element(item, ['predictions', 'entities'])
    ents = [ent for ent in ents if ent['label'] in ['FECHA_RESOLUCION', 'HORA_DE_INICIO', 'HORA_DE_CIERRE']]
    print(ents)
    print('---')
    render(item)
